# Mouse Lung (Stereo-Seq)

In [33]:
import anndata
import scanpy as sc
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
root_dir = os.path.join(os.getcwd(), '..')
if root_dir not in sys.path:
    sys.path.append(root_dir)
import spider
from spider import SPIDER
op=SPIDER()

In [34]:
out_f = '../data/mouse_lung/E5_meta/'
adata = anndata.read_h5ad(f'{out_f}adata.h5ad')
idata = anndata.read_h5ad(f'{out_f}idata.h5ad')
result = pd.read_csv(f'{out_f}SOMDE.csv', index_col=0)
idata.uns['SOMDE'] = result
svi_df, svi_df_strict = op.svi.combine_SVI(idata,threshold=0.01)
adata, idata

Using the results from SVI identification methods: ['SOMDE' 'SpatialDE' 'SPARKX' 'nnSVG' 'scGCO' 'gearyC' 'moranI']
37/1903 SVIs identified (threshold=0.01).


(AnnData object with n_obs × n_vars = 38634 × 25687
     obs: 'annotation', 'row', 'col'
     var: 'gene_ids', 'feature_types'
     uns: 'gearyC', 'spatial_neighbors'
     obsm: 'spatial'
     layers: 'raw_counts'
     obsp: 'spatial_connectivities', 'spatial_distances',
 AnnData object with n_obs × n_vars = 76818 × 1903
     obs: 'A', 'B', 'A_row', 'A_col', 'B_row', 'B_col', 'A_label', 'B_label', 'A_label_int', 'B_label_int', 'label_1', 'label_2', 'label_int', 'label', 'row', 'col', 'dist', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'n_genes', 'total_count', 'som_node'
     var: 'is_svi', 'label', 'pattern_membership_0', 'pattern_membership_1', 'pattern_membership_2', 'pattern_membership_3', 'pattern_membership_4', 'pattern_membership_5', 'pattern_membership_6'
     uns: 'SPARKX', 'SpatialDE', 'cell_meta', 'gearyC', 'lr_meta', 'moranI', 'nnSVG', 'nnSVG_time', 'scGCO', 'scGCO_time', 'SOMDE'
     obsm: 'pattern_score', 'spatial'
     varm: 'svi

In [4]:
out_f = '../data/mouse_lung/D2_meta/'
# adata = anndata.read_h5ad(f'{out_f}adata.h5ad')
idata = anndata.read_h5ad(f'{out_f}idata.h5ad')
idata.to_df().to_csv(f'/home/lishiying/workspace/08-causality/input/2023_4_7_input/D2_idata_count.csv')


In [44]:
idata

AnnData object with n_obs × n_vars = 75514 × 1911
    obs: 'A', 'B', 'A_row', 'A_col', 'B_row', 'B_col', 'A_label', 'B_label', 'A_label_int', 'B_label_int', 'label_1', 'label_2', 'label_int', 'label', 'row', 'col', 'dist', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'n_genes', 'total_count', 'som_node'
    var: 'is_svi', 'label', 'pattern_membership_0', 'pattern_membership_1', 'pattern_membership_2', 'pattern_membership_3'
    uns: 'SPARKX', 'SpatialDE', 'cell_meta', 'gearyC', 'lr_meta', 'moranI', 'nnSVG', 'nnSVG_time', 'scGCO', 'scGCO_time'
    obsm: 'pattern_score', 'spatial'
    obsp: 'spatial_connectivities', 'spatial_distances'

In [54]:
adata.uns['moranI']=pd.read_csv(f'{out_f}svg_moranI.csv',index_col=0)

In [46]:
adata

AnnData object with n_obs × n_vars = 37989 × 25399
    obs: 'annotation', 'row', 'col'
    var: 'gene_ids', 'feature_types'
    uns: 'gearyC', 'spatial_neighbors', 'moranI'
    obsm: 'spatial'
    layers: 'raw_counts'
    obsp: 'spatial_connectivities', 'spatial_distances'

In [55]:
fn='mouse_lung_stereoseq_E5'
cluster_key='annotation'

In [56]:
result = pd.read_csv(f'{out_f}SOMDE.csv', index_col=0)
idata.uns['SOMDE'] = result

In [ ]:
svi_df, svi_df_strict = op.svi.combine_SVI(idata,threshold=0.01)
op.svi.eva_SVI(idata, svi_df_strict)
plt.savefig(f'../figures/{fn}_metric.png')
plt.close()
merged_df,lri_pw_list,gene_lr_list,gene_pw_list = op.vis.svg_svi_relation(adata, idata, title=fn.replace('_',' '), is_human=False)
plt.savefig(f'../figures/{fn}_intersect.png', dpi=600,bbox_inches='tight')
idata.var[[f'pattern_correlation_{x}' for x in range(idata.obsm['pattern_score'].shape[1])]] = 0
corr_df=pd.concat([idata[:,idata.var['is_svi']==1].to_df(),pd.DataFrame(idata.obsm['pattern_score'],index=idata.obs_names)],axis=1).corr().loc[idata[:,idata.var['is_svi']==1].var_names, range(idata.obsm['pattern_score'].shape[1])]
idata.var.loc[idata[:,idata.var['is_svi']==1].var_names, [f'pattern_correlation_{x}' for x in range(idata.obsm['pattern_score'].shape[1])]] = corr_df.to_numpy()
op.vis.pattern_LRI(idata,show_SVI=10)
plt.tight_layout()
plt.savefig(f'../figures/{fn}_patterns.png', dpi=600,bbox_inches='tight')
plt.close()
merged_df_edge, arr = op.er.pattern_enrichment_edge(idata,is_human=False)
merged_df_node, arr = op.er.pattern_enrichment_node(idata,is_human=False)
op.vis.enrichment(merged_df_node,x_key='label', cutoff=0.01, top_term=20,save=f'../figures/{fn}_enrichment_node.png')
op.vis.enrichment(merged_df_edge,x_key='label', cutoff=0.05, top_term=20,save=f'../figures/{fn}_enrichment_edge.png')
adata, adata_lri, adata_pattern = op.cell_transform(idata, adata, label=cluster_key)
sc.pl.rank_genes_groups_dotplot(adata_pattern, n_genes=5, standard_scale='var', cmap='plasma',show=False)
plt.savefig(f'../figures/{fn}_enrichment_pattern.png', bbox_inches="tight",dpi=300)
sc.pl.rank_genes_groups_dotplot(adata_lri, n_genes=5, standard_scale='var', cmap='plasma',show=False)
plt.savefig(f'../figures/{fn}_enrichment_SVI.png', bbox_inches="tight",dpi=300)

In [29]:
out_f = '../data/mouse_lung/E5_meta/'
adata = anndata.read_h5ad(f'{out_f}adata.h5ad')
idata = anndata.read_h5ad(f'{out_f}idata.h5ad')
fn='mouse_lung_stereoseq_E5'
cluster_key='annotation'
result = pd.read_csv(f'{out_f}SOMDE.csv', index_col=0)
idata.uns['SOMDE'] = result
adata, adata_lri, adata_pattern = op.cell_transform(idata, adata, label=cluster_key)
adata.obsm['spatial']=adata.obsm['spatial'].to_numpy().astype(int)
# sc.pl.rank_genes_groups_dotplot(adata_lri, n_genes=5, standard_scale='var', cmap='plasma',show=False)

Added key cell_pattern in idata.uns
Added key cell_score in idata.uns
Added key interaction_pattern, interaction_score in adata.obsm
Added key rank_interaction_score_groups, rank_interaction_pattern_groups in adata.uns


In [18]:
dummy=pd.get_dummies(adata.obs['annotation'])
dummy['exp']=adata.to_df()['Cyp2f2']
bronchioli = adata.obs['annotation'].unique()[dummy.corr().loc['exp', adata.obs['annotation'].unique()].argmax()]
adata.obs['Bronchioli']=(adata.obs['annotation']==bronchioli).astype(int)
df_plot = adata.obs
df_plot[['Chad_Itgb1', 'Cd14_Itgb1', 'Cd14_Tlr3']]=adata_lri.to_df()[['Chad_Itgb1', 'Cd14_Itgb1', 'Cd14_Tlr3']]
df_plot[['x','y']]=adata.obsm['spatial']
plt.figure(figsize=(10, 3))
count=1
for i in ['Bronchioli', 'Chad_Itgb1', 'Cd14_Itgb1', 'Cd14_Tlr3']:
    plt.subplot(1, 4, count)
    im=plt.scatter(df_plot['x'], df_plot['y'], c=df_plot[i], s=1, cmap='plasma')
    plt.axis('equal')
    plt.axis('off')
    plt.title(i)
    plt.colorbar(im,fraction=0.046, pad=0.04)
    count += 1
plt.tight_layout()
plt.savefig(f'../figures/{fn}_Bronchioli.png', dpi=600,bbox_inches='tight')
plt.close()

In [30]:
df_plot = adata.obs
df_plot[['Col1a1_Itgb1', 'Col1a1_Cd93', 'Col1a2_Itgb1']]=adata_lri.to_df()[['Col1a1_Itgb1', 'Col1a1_Cd93', 'Col1a2_Itgb1']]
# df_plot['Nr5a1']=adata.to_df()['Nr5a1"']
df_plot[['x','y']]=adata.obsm['spatial']
plt.figure(figsize=(10, 3))
count=1
for i in ['Col1a1_Itgb1', 'Col1a1_Cd93', 'Col1a2_Itgb1']:
    plt.subplot(1, 4, count)
    im=plt.scatter(df_plot['x'], df_plot['y'], c=df_plot[i], s=1, cmap='plasma')
    plt.axis('equal')
    plt.axis('off')
    plt.title(i)
    plt.colorbar(im,fraction=0.046, pad=0.04)
    count += 1
plt.tight_layout()
plt.savefig(f'../figures/{fn}_AD.png', dpi=600,bbox_inches='tight')
plt.close()